<a href="https://colab.research.google.com/github/galib96/drug-discovery-ml/blob/main/Bioinformatics_Drug_Discovery_Using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chembl-webresource-client

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 636 kB 10.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np

from chembl_webresource_client.new_client import new_client

In [ ]:
# target search for Dengue virus

target = new_client.target
target_query = target.search("Dengue virus")
target_df = pd.DataFrame.from_dict(target_query)

target_df

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Dengue virus,Dengue virus,24.0,False,CHEMBL613757,[],ORGANISM,12637
1,[],Dengue virus 1,Dengue virus 1,21.0,False,CHEMBL613360,[],ORGANISM,11053
2,[],Dengue virus 2,Dengue virus 2,21.0,False,CHEMBL613966,[],ORGANISM,11060
3,[],Dengue virus 3,Dengue virus 3,21.0,False,CHEMBL612717,[],ORGANISM,11069
4,[],Dengue virus 4,Dengue virus type 4,18.0,False,CHEMBL613728,[],ORGANISM,11070
...,...,...,...,...,...,...,...,...,...
163,"[{'xref_id': 'P10274', 'xref_name': None, 'xre...",Human T-cell leukemia virus 1 (strain Japan AT...,Human T-cell leukemia virus type I protease,4.0,False,CHEMBL3346,"[{'accession': 'P10274', 'component_descriptio...",SINGLE PROTEIN,11926
164,[],Human immunodeficiency virus type 1 (RF/HAT IS...,Human immunodeficiency virus type 1 (RF/HAT IS...,4.0,False,CHEMBL613498,[],ORGANISM,11701
165,"[{'xref_id': 'Q69422', 'xref_name': None, 'xre...",Hepatitis GB virus B,Hepatitis GB virus B NS5B RNA-dependent RNA po...,4.0,False,CHEMBL5981,"[{'accession': 'Q69422', 'component_descriptio...",SINGLE PROTEIN,39113
166,[],Homo sapiens,Sharpin/RBCK1/RNF31,4.0,False,CHEMBL4296109,"[{'accession': 'Q9H0F6', 'component_descriptio...",PROTEIN COMPLEX,9606


In [ ]:
target_df[(target_df['target_type']=='SINGLE PROTEIN') & (target_df['organism']=='Dengue virus')]

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
6,[],Dengue virus,Nonstructural protein 5,14.0,False,CHEMBL4295629,"[{'accession': 'V5TFZ2', 'component_descriptio...",SINGLE PROTEIN,12637


In [ ]:
selected_target = target_df.target_chembl_id[6]
selected_target

'CHEMBL4295629'

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')

In [ ]:
df = pd.DataFrame.from_dict(res)
df.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,18297472,[],CHEMBL4123978,Inhibition of Dengue virus 4 NS5 full length R...,B,None,None,BAO_0000190,BAO_0000357,single protein format,N#C[C@]1(O)C(n2ccc3c(N)ncnc32)O[C@H](COP(=O)(O...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '12.18', 'le': '0.27', 'lle': '7.60', ...",CHEMBL4126343,None,CHEMBL4126343,6.47,False,http://www.openphacts.org/units/Nanomolar,3037407,=,1,True,=,None,IC50,nM,None,340.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,0.34
1,None,18297475,[],CHEMBL4123975,Inhibition of Dengue virus 2 NS5 full length R...,B,None,None,BAO_0000190,BAO_0000357,single protein format,[N-]=[N+]=N[C@]1(COP(=O)(O)OP(=O)(O)OP(=O)(O)O...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '12.29', 'le': '0.28', 'lle': '8.01', ...",CHEMBL3417270,None,CHEMBL3417270,6.44,False,http://www.openphacts.org/units/Nanomolar,3037406,=,1,True,=,None,IC50,nM,None,360.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,0.36
2,None,18297488,[],CHEMBL4123982,Inhibition of Dengue virus RdRp activity using...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)OP...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '10.64', 'le': '0.24', 'lle': '7.41', ...",CHEMBL521487,None,CHEMBL521487,5.30,False,http://www.openphacts.org/units/Nanomolar,3037397,=,1,True,=,None,IC50,nM,None,5000.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,5.0
3,None,18297489,[],CHEMBL4123982,Inhibition of Dengue virus RdRp activity using...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '11.41', 'le': '0.26', 'lle': '8.30', ...",CHEMBL4127092,None,CHEMBL4127092,5.80,False,http://www.openphacts.org/units/Nanomolar,3037398,=,1,True,=,None,IC50,nM,None,1600.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,1.6
4,None,18297490,[],CHEMBL4123982,Inhibition of Dengue virus RdRp activity using...,B,None,None,BAO_0000190,BAO_0000357,single protein format,CC#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '10.91', 'le': '0.24', 'lle': '7.81', ...",CHEMBL4127819,None,CHEMBL4127819,5.70,False,http://www.openphacts.org/units/Nanomolar,3037399,=,1,True,=,None,IC50,nM,None,2000.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,2.0


In [ ]:
df.standard_type.unique()

array(['IC50'], dtype=object)

In [ ]:
df.to_csv('bioactivity_data_dengue.csv', index=False)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
#!mkdir '/content/gdrive/My Drive/Colab Notebooks/Bioinformatics Project/data'

In [ ]:
#!cp bioactivity_data_dengue.csv '/content/gdrive/My Drive/Colab Notebooks/Bioinformatics Project/data'

In [ ]:
df_clean = df[df['standard_value'].notna()]
df_clean = df_clean[df_clean['canonical_smiles'].notna()]
df_clean.drop_duplicates(['canonical_smiles'], inplace = True)

print(df_clean.shape)
df_clean.head()

(62, 45)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,18297472,[],CHEMBL4123978,Inhibition of Dengue virus 4 NS5 full length R...,B,None,None,BAO_0000190,BAO_0000357,single protein format,N#C[C@]1(O)C(n2ccc3c(N)ncnc32)O[C@H](COP(=O)(O...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '12.18', 'le': '0.27', 'lle': '7.60', ...",CHEMBL4126343,None,CHEMBL4126343,6.47,False,http://www.openphacts.org/units/Nanomolar,3037407,=,1,True,=,None,IC50,nM,None,340.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,0.34
1,None,18297475,[],CHEMBL4123975,Inhibition of Dengue virus 2 NS5 full length R...,B,None,None,BAO_0000190,BAO_0000357,single protein format,[N-]=[N+]=N[C@]1(COP(=O)(O)OP(=O)(O)OP(=O)(O)O...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '12.29', 'le': '0.28', 'lle': '8.01', ...",CHEMBL3417270,None,CHEMBL3417270,6.44,False,http://www.openphacts.org/units/Nanomolar,3037406,=,1,True,=,None,IC50,nM,None,360.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,0.36
2,None,18297488,[],CHEMBL4123982,Inhibition of Dengue virus RdRp activity using...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)OP...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '10.64', 'le': '0.24', 'lle': '7.41', ...",CHEMBL521487,None,CHEMBL521487,5.30,False,http://www.openphacts.org/units/Nanomolar,3037397,=,1,True,=,None,IC50,nM,None,5000.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,5.0
3,None,18297489,[],CHEMBL4123982,Inhibition of Dengue virus RdRp activity using...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '11.41', 'le': '0.26', 'lle': '8.30', ...",CHEMBL4127092,None,CHEMBL4127092,5.80,False,http://www.openphacts.org/units/Nanomolar,3037398,=,1,True,=,None,IC50,nM,None,1600.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,1.6
4,None,18297490,[],CHEMBL4123982,Inhibition of Dengue virus RdRp activity using...,B,None,None,BAO_0000190,BAO_0000357,single protein format,CC#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O...,None,None,CHEMBL4118212,Bioorg Med Chem Lett,2018,"{'bei': '10.91', 'le': '0.24', 'lle': '7.81', ...",CHEMBL4127819,None,CHEMBL4127819,5.70,False,http://www.openphacts.org/units/Nanomolar,3037399,=,1,True,=,None,IC50,nM,None,2000.0,CHEMBL4295629,Dengue virus,Nonstructural protein 5,12637,None,None,IC50,uM,UO_0000065,None,2.0


In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df_activity = df_clean[selection]

df_activity.head()

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL4126343,N#C[C@]1(O)C(n2ccc3c(N)ncnc32)O[C@H](COP(=O)(O...,340.0
1,CHEMBL3417270,[N-]=[N+]=N[C@]1(COP(=O)(O)OP(=O)(O)OP(=O)(O)O...,360.0
2,CHEMBL521487,C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)OP...,5000.0
3,CHEMBL4127092,C#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)...,1600.0
4,CHEMBL4127819,CC#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O...,2000.0


In [ ]:
def activity_classifier(sv):

    if float(sv) >= 10000:
        return "inactive"
    elif float(sv) <= 1000:
        return "active"
    else:
        return "intermediate"

In [ ]:
df_activity['class'] = df_activity['standard_value'].apply(activity_classifier)

df_activity.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL4126343,N#C[C@]1(O)C(n2ccc3c(N)ncnc32)O[C@H](COP(=O)(O...,340.0,active
1,CHEMBL3417270,[N-]=[N+]=N[C@]1(COP(=O)(O)OP(=O)(O)OP(=O)(O)O...,360.0,active
2,CHEMBL521487,C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)OP...,5000.0,intermediate
3,CHEMBL4127092,C#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)...,1600.0,intermediate
4,CHEMBL4127819,CC#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O...,2000.0,intermediate


In [ ]:
df_activity.to_csv("bioactivity_dengue_preprocessed_data.csv", index=False)

!cp bioactivity_dengue_preprocessed_data.csv '/content/gdrive/My Drive/Colab Notebooks/Bioinformatics Project/data'

In [ ]:
!pip install rdkit-pypi

     |████████████████████████████████| 19.7 MB 1.4 MB/s 


In [ ]:
df_proc = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Bioinformatics Project/data/bioactivity_dengue_preprocessed_data.csv")
df_proc.head()

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL4126343,N#C[C@]1(O)C(n2ccc3c(N)ncnc32)O[C@H](COP(=O)(O...,340.0,active
1,CHEMBL3417270,[N-]=[N+]=N[C@]1(COP(=O)(O)OP(=O)(O)OP(=O)(O)O...,360.0,active
2,CHEMBL521487,C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)OP...,5000.0,intermediate
3,CHEMBL4127092,C#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)...,1600.0,intermediate
4,CHEMBL4127819,CC#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O...,2000.0,intermediate


##Lipinski's descriptors

Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [11]:
# calculate descriptors and lipinski values

df_proc['canonical_smiles_mol'] = df_proc['canonical_smiles'].apply(Chem.MolFromSmiles)
df_proc['mol_wt'] = df_proc['canonical_smiles_mol'].apply(Descriptors.MolWt)
df_proc['mol_logp'] = df_proc['canonical_smiles_mol'].apply(Descriptors.MolLogP)
df_proc['num_H_donors'] = df_proc['canonical_smiles_mol'].apply(Lipinski.NumHDonors)
df_proc['num_H_acceptors'] = df_proc['canonical_smiles_mol'].apply(Lipinski.NumHAcceptors)

df_proc.head()

,molecule_chembl_id,canonical_smiles,standard_value,class,canonical_smiles_mol,mol_wt,mol_logp,num_H_donors,num_H_acceptors
0,CHEMBL4126343,N#C[C@]1(O)C(n2ccc3c(N)ncnc32)O[C@H](COP(=O)(O...,340.0,active,<rdkit.Chem.rdchem.Mol object at 0x7f576d009800>,531.204,-1.13022,7,14
1,CHEMBL3417270,[N-]=[N+]=N[C@]1(COP(=O)(O)OP(=O)(O)OP(=O)(O)O...,360.0,active,<rdkit.Chem.rdchem.Mol object at 0x7f576d009850>,524.169,-1.57400,7,14
2,CHEMBL521487,C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)OP...,5000.0,intermediate,<rdkit.Chem.rdchem.Mol object at 0x7f576d0098f0>,498.167,-2.11080,7,12
3,CHEMBL4127092,C#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O)...,1600.0,intermediate,<rdkit.Chem.rdchem.Mol object at 0x7f576d009940>,508.162,-2.49750,7,12
4,CHEMBL4127819,CC#C[C@@]1(O)[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O...,2000.0,intermediate,<rdkit.Chem.rdchem.Mol object at 0x7f576d009a30>,522.189,-2.10740,7,12
